# Defining our data set
Our data set is composed of 700 poses samples
Each Pose has 30 features.
Poses are split in 7 categories. The purpose of the network will be to predict the correct category for a given sample.
We’ll use 600 samples for training and 100 samples for validation.
We’ll use a batch size of 10 for training and validation

In [2]:
import OpenSSL.SSL
import mxnet as mx
import numpy as np
import logging

logging.basicConfig(level=logging.INFO)
sample_count = 700
train_count = 600
valid_count = sample_count - train_count
feature_count = 30
category_count = 7
batch=10

The categories for these 700 poses samples are represented as integers in the 0–6 range for the sequentially-generated data, 
beacause these data are generated sequentially we need to shuffle them using random.permutation.

In [3]:
# pose Data
poses = np.loadtxt('poses.csv', delimiter=',')
# shuffle data
idx = np.random.permutation(sample_count)

poses_shuffled = []
for i in range(sample_count):
    poses_shuffled.append(poses[idx[i]])

Y = np.zeros([sample_count])
for i in range(0,sample_count):
    Y[i] = i/100
    
y_shuffled = []
for i in range(sample_count):
    y_shuffled.append(Y[idx[i]])
    
poses_shuffled = mx.nd.array(np.asarray(poses_shuffled))
y_shuffled =  mx.nd.array(np.asarray(y_shuffled))
print poses_shuffled.shape
print y_shuffled.shape

(700L, 30L)
(700L,)


# Splitting the data set
Next, we’re splitting the data set 6 to 1 for training and validation. We use the NDArray.crop function to do this. Here, the data set is completely random, so we can use the top 600 sample for training and the bottom 100 sample for validation.

In [4]:
X_train = mx.nd.crop(poses_shuffled, begin=(0,0), end=(train_count,feature_count))
X_valid = mx.nd.crop(poses_shuffled, begin=(train_count,0), end=(sample_count,feature_count))
Y_train = y_shuffled[0:train_count]
Y_valid = y_shuffled[train_count:sample_count]

# Building the network

In [5]:
data = mx.sym.Variable('data')
fc1 = mx.sym.FullyConnected(data, name='fc1', num_hidden=64)
relu1 = mx.sym.Activation(fc1, name='relu1', act_type="relu")
fc2 = mx.sym.FullyConnected(relu1, name='fc2', num_hidden=category_count)
out = mx.sym.SoftmaxOutput(fc2, name='softmax')
mod = mx.mod.Module(out)

# Building the data iterator

In [6]:
train_iter = mx.io.NDArrayIter(data=X_train,label=Y_train,batch_size=batch)

# Training the model

In [7]:
# bind the input symbol to the actual data set (samples and labels). 
mod.bind(data_shapes=train_iter.provide_data, label_shapes=train_iter.provide_label)
# initializer
mod.init_params(initializer=mx.init.Xavier(magnitude=2.))
mod.init_optimizer(optimizer='sgd', optimizer_params=(('learning_rate', 0.1), ))
mod.fit(train_iter, num_epoch=1000)

/Users/abdu307/anaconda3/envs/python2/lib/python2.7/site-packages/mxnet/module/base_module.py:464: UserWarning: Parameters already initialized and force_init=False. init_params call ignored.
  allow_missing=allow_missing, force_init=force_init)
INFO:root:Epoch[0] Train-accuracy=0.211667
INFO:root:Epoch[0] Time cost=0.080
INFO:root:Epoch[1] Train-accuracy=0.295000
INFO:root:Epoch[1] Time cost=0.055
INFO:root:Epoch[2] Train-accuracy=0.381667
INFO:root:Epoch[2] Time cost=0.051
INFO:root:Epoch[3] Train-accuracy=0.428333
INFO:root:Epoch[3] Time cost=0.052
INFO:root:Epoch[4] Train-accuracy=0.483333
INFO:root:Epoch[4] Time cost=0.052
INFO:root:Epoch[5] Train-accuracy=0.586667
INFO:root:Epoch[5] Time cost=0.053
INFO:root:Epoch[6] Train-accuracy=0.655000
INFO:root:Epoch[6] Time cost=0.050
INFO:root:Epoch[7] Train-accuracy=0.690000
INFO:root:Epoch[7] Time cost=0.053
INFO:root:Epoch[8] Train-accuracy=0.745000
INFO:root:Epoch[8] Time cost=0.051
INFO:root:Epoch[9] Train-accuracy=0.776667
INFO:root:

# Validating the model

In [8]:
pred_iter = mx.io.NDArrayIter(data=X_valid,label=Y_valid, batch_size=batch)
pred_count = valid_count
correct_preds = total_correct_preds = 0
for preds, i_batch, batch in mod.iter_predict(pred_iter):
    label = batch.label[0].asnumpy().astype(int)
    pred_label = preds[0].asnumpy().argmax(axis=1)
    correct_preds = np.sum(pred_label==label)
    total_correct_preds = total_correct_preds + correct_preds
print('Validation accuracy: %2.2f' % (1.0*total_correct_preds/pred_count))

Validation accuracy: 0.99


# save a model to pose-symbol.json and pose-1000.params

In [9]:
import os
filename = os.getcwd()+"/pose"
mod.save_checkpoint(filename,1000)

INFO:root:Saved checkpoint to "/Users/abdu307/deeplens-simon-says/development/pose-1000.params"


# leading and  Testing the model 

In [10]:
import os
import mxnet as mx
import numpy as np
import logging
logging.basicConfig(level=logging.INFO)
sample_count = 7*20
train_count = 4*20
valid_count = sample_count - train_count
feature_count = 15*2
category_count = 7
batch=10

X_pred = mx.nd.zeros((10,feature_count))
Y_pred = mx.nd.empty((10,))

pred_iter = mx.io.NDArrayIter(data=X_pred,label=Y_pred, batch_size=batch)

filename = os.getcwd()+"/pose"
sym, arg_params, aux_params = mx.model.load_checkpoint(filename, 1000)

new_model = mx.mod.Module(symbol=sym)
new_model.bind(pred_iter.provide_data, pred_iter.provide_label)
new_model.set_params(arg_params, aux_params)


In [12]:
for i in range(0,700,100):
    pose = mx.nd.array(poses[i])

    X_pred = mx.nd.empty((10,feature_count))
    Y_pred = mx.nd.empty((10,))

    X_pred[0] = pose

    #print(X_pred)
    pred_iter = mx.io.NDArrayIter(data=X_pred,label=Y_pred, batch_size=batch)

    a = new_model.predict(pred_iter)[0]
    a= list(a.asnumpy())
    print max(a)
    p = a.index(max(a))
    print ("real pose:",p,"   predicted pose:",Y[i])


0.999269
('real pose:', 0, '   predicted pose:', 0.0)
0.999755
('real pose:', 1, '   predicted pose:', 1.0)
0.997491
('real pose:', 2, '   predicted pose:', 2.0)
0.976469
('real pose:', 3, '   predicted pose:', 3.0)
0.999867
('real pose:', 4, '   predicted pose:', 4.0)
0.999839
('real pose:', 5, '   predicted pose:', 5.0)
1.0
('real pose:', 6, '   predicted pose:', 6.0)
